<a href="https://colab.research.google.com/github/salemsama/nn-zero-to-hero/blob/main/makemore1/makemore_part1_E01_trigram_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one.
# Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

# result:
# Loss = 2.2119739055633545 compared to 2.476470470428467(karpathy) or in my case it was 2.5839083194732666
# samples sounds more like names: junide. jakasid. prelay. adin. kairritoper.

import torch
N = torch.zeros((729, 27), dtype=torch.int32)

words = open("names.txt", 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [5]:
# this is trigram and exercise on bigram
# counting version

for w in words:
  #we add 2 dots because we need a context (learned this idea from the video after this aka makemore 2)
  chs = ['.','.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1*27 + ix2, ix3] += 1

P = (N+1).float()
P /= P.sum(1, keepdims=True)
g = torch.Generator().manual_seed(2147483647)

In [6]:
#sampling
for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    p = P[ix1*27 + ix2]
    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break

  print(''.join(out))

junide.
jakasid.
prelay.
adin.
kairritoper.


In [7]:
# calculate loss
log_likelihood = 0.0
n = 0

for w in words:
  chs = ['.','.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[ix1*27 + ix2, ix3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n +=1

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-504653.)
nll=tensor(504653.)
2.2119739055633545
